In [1]:
import re
import os
import numpy as np
import pandas as pd
from pprint import pprint
from operator import itemgetter

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
import nltk
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

musings_path = 'sam_musings/'
musings = os.listdir(musings_path)

# Import Dataset
print('Loading musings...')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
musings = '\n'.join([open(musings_path + musing).read() for musing in musings])
musing_sentences = tokenizer.tokenize(musings)
df = pd.DataFrame([{'content': sentence} for sentence in musing_sentences])

# Convert to list
data = df.content.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        # deacc=True removes punctuations

data_words = list(sent_to_words(data))

# Build the bigram and trigram models
print('Making bigrams...')
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

print('Making trigrams...')
#trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
print('Modding n-grams...')
bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        try:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        except TypeError:
            if len(sent) == 0:
                print 'Null bigram found'
            else:
                print 'something is wrong with bigram: {}'.format(sent)
    return texts_out

# Remove Stop Words
print('Destroying stop words...')
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
print('Forming bigrams...')
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
print('Building spacy NLP parser...')
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
print('Lemmatizing bigrams...')
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
print('Building dictionary...')
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
print('Making corpus...')
corpus = [id2word.doc2bow(text) for text in texts]

topic_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 200]
lda_models = [] * len(topic_sizes)
coherence_model_ldas = [] * len(topic_sizes)
coherence_lda = np.zeros(len(topic_sizes))

# Build LDA model
print('Generating LDA Model...')
#for index, topics in enumerate(topic_sizes):
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                   id2word=id2word,
                                                   num_topics=250,
                                                   random_state=100,
                                                   update_every=1,
                                                   chunksize=100,
                                                   passes=10,
                                                   alpha='auto',
                                                   per_word_topics=True)
    
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Generate mallet topic
mallet_path = '../mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=150, id2word=id2word)

"""
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """"""
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """"""
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


limit=150; start=50; step=10;

# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start, limit=limit, step=step)

# Show graph
x = range(start, limit, step)

# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
"""
    
# Select the model and print the topics
#optimal_model = model_list[np.argmax(coherence_values)]
optimal_model = ldamallet
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic


Loading musings...
Making bigrams...
Making trigrams...
Modding n-grams...
Destroying stop words...
Forming bigrams...
Building spacy NLP parser...
Lemmatizing bigrams...
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
N

Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigra

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,19,0.0581,"faculty, member, hire, senior, tenure, henry, ...","On Tuesday at lunch, I'm part of a small group..."
1,1,90,0.0238,"write, essay, short, sit, recommendation_lette...",I think this is one of those times that I need...
2,2,143,0.0385,"find, hope, easy, tomorrow, trouble, continue,...",I hope I don't get myself in too much trouble.
3,3,4,0.0432,"college, institution, education, grinnell, lib...",Shared governance is a core aspect of higher e...
4,4,75,0.0296,"faculty, member, staff, salary, handbook, earl...",Its basic premise should be simple: The facult...
5,5,121,0.0404,"make, difference, decision, choice, bad, huge,...","That is, they make decisions collaboratively w..."
6,6,19,0.0710,"faculty, member, hire, senior, tenure, henry, ...","At times, collaboration may take the form on c..."
7,7,138,0.0424,"policy, grinnell, president, posse, relationsh...",Shared governance usually requires some form o...
8,8,85,0.0394,"committee, serve, sepc, purpose, stand, repres...","That is, because it is complicated to delibera..."
9,9,40,0.0444,"group, share, include, associate, acm, ethic, ...",Those representative groups then have a respon...


In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
sent_topics_sorteddf_mallet['Text'][1]

In [ ]:
def getTopicForQuery(question, model=lda_model):
    temp = question.lower()

    words = re.findall(r'\w+', temp, flags = re.UNICODE | re.LOCALE)
    
    important_words = [word for word in simple_preprocess(str(words)) if word not in stop_words]
    
    dictionary = corpora.Dictionary.load('sam_dict.dict')

    ques_vec = dictionary.doc2bow(important_words)

    topic_vec = model[[ques_vec]]

    likely_topic = max(topic_vec[0], key=itemgetter(1))[0]
    
    return likely_topic

In [ ]:
query_dominant_topic = getTopicForQuery("Good morning Sam Bot.", model=optimal_model)
related_sentences = df_dominant_topic[df_dominant_topic['Dominant_Topic'] == query_dominant_topic].reset_index(drop=True)
related_sentences.loc[random.randint(0,len(related_sentences))]['Text']

In [ ]:
id2word.save('sam_dict.dict')

In [ ]:
lda_model.get_topics().shape

In [ ]:
df.loc[4].content

In [ ]:
[[score for topic_id, score in topic] for topic in [doc for doc in lda_corpus]]

In [ ]:
lda_model.print_topic(29)

In [ ]:
from operator import itemgetter
import random

In [2]:
df_dominant_topic.to_csv('sam_dominant_topics.csv')

In [3]:
optimal_model.save('sam_lda_model.mm')

In [4]:
id2word.save('sam_dict.dict')

In [6]:
gensim.models.wrappers.LdaMallet.

AttributeError: 'module' object has no attribute 'models'

In [5]:
help(gensim.models.wrappers.LdaMallet)

Help on class LdaMallet in module gensim.models.wrappers.ldamallet:

class LdaMallet(gensim.utils.SaveLoad, gensim.models.basemodel.BaseTopicModel)
 |  Python wrapper for LDA using `MALLET <http://mallet.cs.umass.edu/>`_.
 |  
 |  Communication between MALLET and Python takes place by passing around data files on disk
 |  and calling Java with subprocess.call().
 |  
 |  Warnings
 |  --------
 |  This is **only** python wrapper for `MALLET LDA <http://mallet.cs.umass.edu/>`_,
 |  you need to install original implementation first and pass the path to binary to ``mallet_path``.
 |  
 |  Method resolution order:
 |      LdaMallet
 |      gensim.utils.SaveLoad
 |      gensim.models.basemodel.BaseTopicModel
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, bow, iterations=100)
 |      Get vector for document(s).
 |      
 |      Parameters
 |      ----------
 |      bow : {list of (int, int), iterable of list of (int, int)}
 |          Document (or corpus)